In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

import pandas as pd
import numpy as np

import requests
import time
import random
from tqdm.notebook import tqdm

import glob

In [2]:
options = Options()

In [3]:
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")

In [4]:
url_filename = 'urls.txt'

In [5]:
if (not url_filename in glob.glob('*.*')):

    driver = webdriver.Chrome("chromedriver.exe", options=options)
    
    url = f'https://www.bookrix.com/books;fantasy,id:52.html'
    driver.get(url)
    time.sleep(5)
    
    book_urls = []
    num_pages = int(driver.find_element(by = 'class name', value = 'pager').text.split('\n')[-1])
    
    
    current_page = 1
    
    while (current_page <= num_pages):
    
        books = driver.find_elements(by = 'class name', value = 'item')
        
        for book in books:
            url = book.find_element(by = 'class name', value = 'word-break').get_attribute('href')
            book_urls.append(url)
        
        pager = driver.find_element(by = 'class name', value = 'pager')
        pager.find_element(by = 'class name', value = 'next').click()
        
        random_wait = random.randrange(5, 15)
        time.sleep(random_wait)
        
        current_page += 1

    with open("urls.txt", 'w') as f:
        for s in book_urls:
            f.write(str(s) + '\n')
    
    driver.close()
            
else:
    with open(url_filename, 'r') as f:
        book_urls = [line.rstrip('\n') for line in f]

In [6]:
books_data = []

In [7]:
columns = ['title', 'author', 'summary', 'category', 'word_count',
           'age_cat', 'num_views', 'num_fav', 'publication_date',
           'tags', 'url', 'download_link']

df_filename = 'books.csv'

if (df_filename in glob.glob('*.*')):
    df_books = pd.read_csv(df_filename, index_col = 0)
else:
    df_books = pd.DataFrame(books_data, columns=columns)

In [8]:
num_to_collect = 200

In [9]:
collected = 0

for url in tqdm(book_urls):
    try:
        if (collected >= num_to_collect):
            break
        if (url in list(df_books['url'])):
            continue
        
        driver = webdriver.Chrome("chromedriver.exe", options=options)
        driver.get(url)
        
        random_wait = random.randrange(10, 20)
        time.sleep(random_wait)
        
        title_box = driver.find_element(by = 'id', value = 'booktitle')
        title = title_box.find_element(by = 'class name', value = 'break-word').text
        author = title_box.find_element(by = 'id', value = 'authortext').text.split(': ')[1]
        summary = driver.find_element(by = 'id', value = 'blurb').text
        
        details = driver.find_element(by = 'id', value = 'details')
        
        details_top = [text.text for text in details.find_elements(by = 'tag name', value = 'li')]
        category = details_top[0]
        word_count = details_top[1]
        age_cat = details_top[2]
        num_views = details_top[3]
        num_fav = details_top[4]
        publication_date = details_top[5].split(': ')[1]
        
        try:
            tags = set(details.find_element(by = 'id', value = 'bookkeywords').text.split('\n')[1].split(', '))
            
        except Exception as e:
            tags = np.nan
        
        download_boxes = driver.find_element(by = 'id', value = 'bookfree')
        
        download_button = download_boxes.find_element(by = 'tag name', value = 'span')
        
        if (not download_button.find_element(by = 'tag name', value = 'a').get_attribute('class').split(' ')[-1] == 'disabled'):
            download_button.click()
            
            random_wait = random.randrange(10, 15)
            time.sleep(random_wait)
            
            download_link = driver.find_element(by = 'class name', value = 'bxmodal-backdrop').find_element(by = 'class name', value = 'modal-body').find_element(by = 'tag name', value = 'a').get_attribute('href')
        else:
            download_link = np.nan
            
        book_info = []
        book_info.append(title)
        book_info.append(author)
        book_info.append(summary)
        book_info.append(category)
        book_info.append(word_count)
        book_info.append(age_cat)
        book_info.append(num_views)
        book_info.append(num_fav)
        book_info.append(publication_date)
        book_info.append(tags)
        book_info.append(url)
        book_info.append(download_link)
        
        books_data.append(book_info)
        
        driver.close()
        
        collected += 1
    except Exception as e:
        pass

  0%|          | 0/3686 [00:00<?, ?it/s]

C:\Users\alcob\AppData\Local\Temp/ipykernel_17044/3657312954.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe", options=options)


In [10]:
df_collected = pd.DataFrame(books_data, columns=columns)
df_collected

,title,author,summary,category,word_count,age_cat,num_views,num_fav,publication_date,tags,url,download_link
0,ALPHA,SHANNON COOTS,"Alpha Blake Malone is a force, and respected b...",Fantasy,3541 Words,Ages 0 and up,3331,123,08-30-2011,"{friend, moon, midnight,... Show more, mate, s...",https://www.bookrix.com/_ebook-shannon-coots-a...,https://www.bookrix.com/Books/Download.html?bo...
1,Cliche?,R.G Teongson,I was rejected- my soulmate. He was suppose to...,Fantasy,2581 Words,Ages 16 and up,2941,94,04-23-2012,"{sex, rejected, friends,... Show more, die, al...",https://www.bookrix.com/_ebook-r-g-teongson-cl...,https://www.bookrix.com/Books/Download.html?bo...
2,Breakfast with a Werewolf,Marissa L. Zepperi,"Hey I'm Missi, and I am a normal girl with nor...",Fantasy,30 Words,Ages 14 and up,2921,61,05-29-2011,NaN,https://www.bookrix.com/_ebook-marissa-l-zeppe...,https://www.bookrix.com/Books/Download.html?bo...
3,Feral Heart Edit,Duron Crejaro,"The World burned, and with it most of life. Th...",Fantasy,9628 Words,Ages 16 and up,2863,23,03-20-2016,NaN,https://www.bookrix.com/_ebook-duron-crejaro-f...,https://www.bookrix.com/Books/Download.html?bo...
4,The Vampire Joy,Alisa Freund,"I know, I know another vampire book. this is a...",Fantasy,22200 Words,Ages 0 and up,2855,236,07-24-2010,{vampire},https://www.bookrix.com/_ebook-alisa-freund-th...,https://www.bookrix.com/Books/Download.html?bo...
...,...,...,...,...,...,...,...,...,...,...,...,...
195,Nightmare Story,Chris Williams,"The harder I tried the more my brain pulsed, t...",Fantasy,5011 Words,Ages 0 and up,699,1,02-27-2011,"{Six, nightmare, Nightmare Story, fire, story,...",https://www.bookrix.com/_ebook-chris-williams-...,https://www.bookrix.com/Books/Download.html?bo...
196,Whats Happening To Me,AJ Joseph,Fey is a orphan not knowing who her real paren...,Fantasy,2475 Words,Ages 0 and up,694,22,06-15-2011,"{love, aliens}",https://www.bookrix.com/_ebook-aj-joseph-whats...,https://www.bookrix.com/Books/Download.html?bo...
197,My High School FairyTail,xomangagirlxo,"Sakura is a smart, pretty girl from a rich fam...",Fantasy,2367 Words,Ages 0 and up,696,23,09-04-2011,"{Romance, Love, Japan, Hosts, anime boy+girl, ...",https://www.bookrix.com/_ebook-xomangagirlxo-m...,https://www.bookrix.com/Books/Download.html?bo...
198,A New Day,Eartha15,I am happy!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!...,Fantasy,3110 Words,Ages 0 and up,694,3,05-17-2012,{Sailor Moon and Dragon Ball Z},https://www.bookrix.com/_ebook-eartha15-a-new-...,https://www.bookrix.com/Books/Download.html?bo...


In [11]:
df_books = df_books.append(df_collected, ignore_index = True)
df_books

,title,author,summary,category,word_count,age_cat,num_views,num_fav,publication_date,tags,url,download_link
0,My Personal Hell,D. Richardson,"For seventeen year old Ailith, grief and p...",Fantasy,168571 Words,Ages 12 and up,48035,422,10-09-2013,"{'cruelty', 'love', 'werewolves', 'torture', '...",https://www.bookrix.com/_ebook-d-richardson-my...,NaN
1,Marked House of Night,Kristin Cast,A girl is a vampire and goes to school. At a v...,Fantasy,1635 Words,Ages 0 and up,25090,413,12-06-2011,NaN,https://www.bookrix.com/_ebook-kristin-cast-ma...,https://www.bookrix.com/Books/Download.html?bo...
2,Pack Gems,"Kuro Kusai, WhiteWolf Aura",Taylor and Kaden fight fang and claw to keep t...,Fantasy,79586 Words,Ages 12 and up,24830,647,04-02-2011,"{'Pack', 'Shapeshifter,... Show more', 'Fantas...",https://www.bookrix.com/_ebook-kuro-kusai-pack...,https://www.bookrix.com/Books/Download.html?bo...
3,SPELLS OF MAGIC,Forest Ostrander,These spells are actual spells. I would highly...,Fantasy,7957 Words,Ages 0 and up,19129,95,03-04-2012,NaN,https://www.bookrix.com/_ebook-forest-ostrande...,https://www.bookrix.com/Books/Download.html?bo...
4,Marked (book 1),Kristin Cast,The House of Night series is set in a world ve...,Fantasy,82309 Words,Ages 6 and up,17210,52,10-28-2014,{'fantasy'},https://www.bookrix.com/_ebook-kristin-cast-ma...,https://www.bookrix.com/Books/Download.html?bo...
...,...,...,...,...,...,...,...,...,...,...,...,...
459,Nightmare Story,Chris Williams,"The harder I tried the more my brain pulsed, t...",Fantasy,5011 Words,Ages 0 and up,699,1,02-27-2011,"{Six, nightmare, Nightmare Story, fire, story,...",https://www.bookrix.com/_ebook-chris-williams-...,https://www.bookrix.com/Books/Download.html?bo...
460,Whats Happening To Me,AJ Joseph,Fey is a orphan not knowing who her real paren...,Fantasy,2475 Words,Ages 0 and up,694,22,06-15-2011,"{love, aliens}",https://www.bookrix.com/_ebook-aj-joseph-whats...,https://www.bookrix.com/Books/Download.html?bo...
461,My High School FairyTail,xomangagirlxo,"Sakura is a smart, pretty girl from a rich fam...",Fantasy,2367 Words,Ages 0 and up,696,23,09-04-2011,"{Romance, Love, Japan, Hosts, anime boy+girl, ...",https://www.bookrix.com/_ebook-xomangagirlxo-m...,https://www.bookrix.com/Books/Download.html?bo...
462,A New Day,Eartha15,I am happy!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!...,Fantasy,3110 Words,Ages 0 and up,694,3,05-17-2012,{Sailor Moon and Dragon Ball Z},https://www.bookrix.com/_ebook-eartha15-a-new-...,https://www.bookrix.com/Books/Download.html?bo...


In [12]:
df_books.to_csv(f'books.csv')

In [13]:
# r = requests.get(download_link, allow_redirects=True)
#open(f'Books/{title}.epub', 'wb').write(r.content)